In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
%matplotlib inline
labels = pd.read_csv("validation.csv", delimiter = ";")

In [4]:
data_train = df.iloc[:,1:]
data_test= test.iloc[:,1:]
data_labels = labels

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train, data_labels, test_size=0.2, random_state=42)

In [6]:

PM2 = y_train.filter(regex='PM2')
PM10 = y_train.filter(regex='PM10')
O3 = y_train.filter(regex= "O3")
NO2= y_train.filter(regex= "NO2")
PM2_test = y_test.filter(regex='PM2')
PM10_test  = y_test.filter(regex='PM10')
O3_test  = y_test.filter(regex= "O3")
NO2_test = y_test.filter(regex= "NO2")

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb 
from time import time
n,m = y_test.shape

y_pred = np.zeros((n,m))
i=0
s = [PM2, PM10, O3,NO2]
clfs = []

params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.01
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 5

plst = list(params.items())
xgtest = xgb.DMatrix(X_test)

num_rounds = 2000
for target in s:
    
    l = len(target.columns)
    col = target.columns[0]

    print col
    xgtrain = xgb.DMatrix(X_train, label=target[col])
    
    
    start = time()
    model = xgb.train(plst, xgtrain, num_rounds)  
    end = time() - start
    print "Fitting the model took %d seconds"%end

    pred = model.predict(xgtest)
    print "MSE : ", mean_squared_error(y_test[col],pred)
    print "R2 : ",r2_score(y_test[col],pred)

MSE :  8.43513213464
R2 :  0.936010335225
